<a href="https://colab.research.google.com/github/Ayan1311819/Fun_Projects/blob/main/f_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 16.1 MB/s eta 0:00:00


In [ ]:
!pip install html2image

In [ ]:
# Step 1: Install Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y  # Fix any dependencies

# Step 2: Install selenium and webdriver-manager if needed
!pip install selenium webdriver-manager


--2025-04-29 09:27:59--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 172.253.117.91, 172.253.117.93, 172.253.117.190, ...
Connecting to dl.google.com (dl.google.com)|172.253.117.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115150800 (110M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 109.82M   240MB/s    in 0.5s    

2025-04-29 09:27:59 (240 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [115150800/115150800]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 126101 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (135.0.7049.114-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Packag

In [ ]:
import os
from docx import Document
from docx.shared import Inches, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
import google.generativeai as genai
from html2image import Html2Image
import time
import tempfile
import shutil
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from IPython.display import Image

In [ ]:
from PIL import Image, ImageChops

def autocrop_image(input_path, output_path, padding=10, max_size=(800, 800)):
    image = Image.open(input_path)
    bg = Image.new(image.mode, image.size, image.getpixel((0, 0)))  # assume top-left pixel is background
    diff = ImageChops.difference(image, bg)
    bbox = diff.getbbox()

    if bbox:
        # padding for not cropping to the perfection
        left = max(bbox[0] - padding, 0)
        top = max(bbox[1] - padding, 0)
        right = min(bbox[2] + padding, image.width)
        bottom = min(bbox[3] + padding, image.height)
        cropped = image.crop((left, top, right, bottom))
        # Resize the cropped image
        cropped.thumbnail(max_size, Image.Resampling.LANCZOS)

        cropped.save(output_path)
    else:
        image.save(output_path)


In [ ]:
GOOGLE_API_KEY = "gemini_API_key"
genai.configure(api_key=GOOGLE_API_KEY)

def get_gemini_response(prompt):
    """Send a prompt to Gemini API and return the response."""
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text

def clean(content):
    content = content.strip()  # remove any extra newlines/spaces
    if content.startswith("```html"):
        content = content[7:]  # remove ```html
    elif content.startswith("```"):
        content = content[3:]  # remove ```
    if content.endswith("```"):
        content = content[:-3]  # remove ending ```
    return content.strip()  # just in case there are spaces/newlines

def generate_code_and_output(aim):
    """Prompt Gemini API for code and output with HTML styling."""
    # Prompt for code in HTML styled like VS Code
    code_prompt = f'''Given the aim: '{aim}', write the corresponding Python code.
Then, create an HTML document that:
- Displays this code using a <pre><code> block to preserve indentation and line breaks exactly as in the original code.
- Styles it to match Visual Studio Code's default dark theme (background: #1e1e1e, default text: #d4d4d4, keywords like 'if', 'else', 'try', 'except', 'print' in #569cd6, strings in #ce9178) with no additional padding or margins beyond the VS Code style.
- Includes syntax highlighting by wrapping keywords and strings in <span> tags with the specified colors.
Return ONLY the raw HTML content—no explanations or comments.'''
    code_html = get_gemini_response(code_prompt)
    code_html=clean(code_html)
    # Prompt for code output in HTML styled like VS Code terminal
    output_prompt = f'''Given the aim: \"{aim}\", please do the following steps in order:
1. Execute the expected Python code with example inputs to simulate a full program run (e.g., for input prompts, use sample values like "Alice" for a name and "20" for an age, and capture all resulting output, including prompts and print statements).
2. Produce an HTML document that renders exactly like a Visual Studio Code integrated terminal, using:
   - Background color: #1e1e1e
   - Text color: #ffffff
   - Monospace font (e.g., Consolas or a similar font)
   - Display each line of exact expected output as it would appear in a terminal, prefixing interactive prompts with “>>> ” (e.g., ">>> Enter your name: Alice") and print statements on new lines without the “>>> ” prefix (e.g., "Alice, you are eligible to vote!").
Return only the raw HTML content—no additional text or explanations.'''
    output_html = get_gemini_response(output_prompt)
    output_html = clean(output_html)
    return code_html, output_html

def save_html_and_screenshot(html_content, filename, screenshot_path):
    # Step 1: Save the HTML content to file
    html_filename = f"{filename}.html"
    with open(html_filename, "w", encoding="utf-8") as f:
        f.write(html_content)

    try:
        # Step 2: Setup Chrome options
        chrome_options = Options()
        chrome_options.add_argument('--headless=new')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--remote-debugging-port=9222')
        chrome_options.add_argument('--window-size=1280x1024')
        chrome_options.binary_location = '/usr/bin/google-chrome'  # Important for Colab

        # Step 3: Start ChromeDriver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        # Step 4: Render the HTML file
        absolute_path = os.path.abspath(html_filename)
        print(f"Rendering HTML from: {absolute_path}")
        driver.get(f"file://{absolute_path}")
        time.sleep(1)  # Wait for page to load

        # Step 5: Take screenshot
        driver.save_screenshot(screenshot_path)
        print(f"Screenshot saved to: {os.path.abspath(screenshot_path)}")

        driver.quit()

        # Step 6 (Optional): Show the screenshot inside Colab
        #display(Image(filename=screenshot_path))

    except Exception as e:
        print(f"Error taking screenshot: {str(e)}")


def create_experiment_doc(section, name, enrollment, aim):

    #Create Word document
    doc = Document()
    for idx, aim in enumerate(aims):
      #Generate code and output HTML using Gemini API
      code_html, output_html = generate_code_and_output(aim)

      # Render HTML and capture screenshots
      code_screenshot = "code_screenshot.png"
      output_screenshot = "output_screenshot.png"
      save_html_and_screenshot(code_html, "code", code_screenshot)
      save_html_and_screenshot(output_html, "output", output_screenshot)
    # Access header
      if idx==0:
        section_obj = doc.sections[0]
        header = section_obj.header

        # Add paragraph to header
        header_para = header.add_paragraph()

        # Set tab stops: left, center, and right
        tab_stops = header_para.paragraph_format.tab_stops
        page_width = section_obj.page_width - section_obj.left_margin - section_obj.right_margin

        tab_stops.add_tab_stop(Inches(0))                         # Left (default, optional)
        tab_stops.add_tab_stop(page_width // 2)                    # Center
        tab_stops.add_tab_stop(page_width)                        # Right

        # Add text with tabs to split into 3 parts
        header_para.text = f"{section}\t{name}\t{enrollment}"

      # EXP
      exp_para = doc.add_paragraph()
      exp_para.alignment = WD_ALIGN_PARAGRAPH.CENTER  # Center align the paragraph
      exp_run = exp_para.add_run("EXPERIMENT: ")
      exp_run.bold = True
      exp_run.font.color.rgb = RGBColor(0, 0, 0)
      exp_para.add_run(str(idx+1))

      # AIM
      aim_para = doc.add_paragraph()
      aim_run = aim_para.add_run("AIM: ")
      aim_run.bold = True
      aim_run.font.color.rgb = RGBColor(0, 0, 0)
      aim_para.add_run(aim)

      # Theory Placeholder
      theory_heading = doc.add_heading("Theory", level=1)
      for run in theory_heading.runs:
          run.font.color.rgb = RGBColor(0, 0, 0)
      doc.add_paragraph()
      doc.add_paragraph()
      doc.add_paragraph()

      # Code Section
      code_heading = doc.add_heading("CODE", level=1)
      for run in code_heading.runs:
          run.font.color.rgb = RGBColor(0, 0, 0)
      if os.path.exists(code_screenshot):
          autocrop_image(code_screenshot, code_screenshot)
          doc.add_picture(code_screenshot, width=Inches(6.0))
      else:
        print("unsuccessfull")
        doc.add_paragraph("[Code screenshot placeholder]")

      # Output Section
      output_heading = doc.add_heading("OUTPUT", level=1)
      for run in output_heading.runs:
          run.font.color.rgb = RGBColor(0, 0, 0)
      if os.path.exists(output_screenshot):
          autocrop_image(output_screenshot, output_screenshot)
          doc.add_picture(output_screenshot, width=Inches(4.0))
      else:
        print("unsuccessfull")
        doc.add_paragraph("[Output screenshot placeholder]")

      if idx != len(aims) - 1:
              doc.add_page_break()
      os.remove(code_screenshot)
      os.remove(output_screenshot)
    # Save the document
    doc.save("python_file_10.docx")
    print("Document created: test_doc.docx")

    # Clean up screenshots
    #if os.path.exists(code_screenshot):
     #   os.remove(code_screenshot)
    #if os.path.exists(output_screenshot):
     #   os.remove(output_screenshot)

if __name__ == "__main__":
    # Sample inputs
    section = input("Enter section (e.g., VIPS-TC): ") or "VIPS-TC"
    name = input("Enter name: ") or "Ayan Choudhary"
    enrollment = input("Enter enrollment number: ") or "072177022722"
    aims = [
    "To write a Python program that prompts the user for their name and age, and prints a personalized message.",
    "To write a Python program that checks if the user is eligible to vote based on their age.",
    "To write a Python program to calculate the factorial of a number using a loop.",
    "To write a Python program that sorts a list of numbers in ascending order.",
    "To write a Python program that reverses a user-provided string.",
    "To write a Python program that calculates the area of a circle using a function.",
    "To write a Python program that defines a class for a car and creates an object with specific attributes.",
    "To write a Python program that reads from one file and writes to another in a different format.",
    "To write a Python program that uses regular expressions to find a specific pattern in a text file.",
    "To write a Python program that divides two numbers and handles exceptions appropriately."
]

    create_experiment_doc(section, name, enrollment, aims)


Enter section (e.g., VIPS-TC): 
Enter name: 
Enter enrollment number: 
Rendering HTML from: /content/code.html
Screenshot saved to: /content/code_screenshot.png
Rendering HTML from: /content/output.html
Screenshot saved to: /content/output_screenshot.png
Rendering HTML from: /content/code.html
Screenshot saved to: /content/code_screenshot.png
Rendering HTML from: /content/output.html
Screenshot saved to: /content/output_screenshot.png
Rendering HTML from: /content/code.html
Screenshot saved to: /content/code_screenshot.png
Rendering HTML from: /content/output.html
Screenshot saved to: /content/output_screenshot.png
Rendering HTML from: /content/code.html
Screenshot saved to: /content/code_screenshot.png
Rendering HTML from: /content/output.html
Screenshot saved to: /content/output_screenshot.png
Rendering HTML from: /content/code.html
Screenshot saved to: /content/code_screenshot.png
Rendering HTML from: /content/output.html
Screenshot saved to: /content/output_screenshot.png
Rendering